# Import

In [ ]:
import time
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, r2_score, mean_absolute_error
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error

#Fungsi ELM

In [ ]:
class ELM:
    def __init__(self, input_size, hidden_size, output_size):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size

        # Generate random input-to-hidden weights and biases
        self.weights = np.random.rand(hidden_size, input_size) / np.sqrt(input_size)
        self.bias_hidden = np.random.rand(hidden_size, 1) / np.sqrt(hidden_size)
        self.beta = 0.2

    def train(self, X_train, y_train):
        Hinit =  X_train @ self.weights.T
        H = self._sigmoid(Hinit)
        Ht = H.T
        Hp = np.linalg.pinv(Ht @ H) @ Ht
        self.beta  = Hp @ y_train


    def predict(self, X):
        Hinit = X @ self.weights.T
        H = self._sigmoid(Hinit)
        output = H @ self.beta
        return output

    def _sigmoid(self, x):
        return 1.0 / (1.0 + np.exp(-x))

def format_rupiah(amount):
    """Format nilai dalam bentuk nominal rupiah."""
    rupiah = "Rp {:,.2f}".format(amount)
    return rupiah

def get_user_input(data_mm):
    print("Masukkan fitur-fitur rumah sesuai kebutuhan anda")

    LT = float(input("Luas Tanah (LT): "))
    LB = float(input("Luas Bangunan (LB): "))
    JKT = int(input("Jumlah Kamar Tidur (JKT): "))
    JKM = int(input("Jumlah Kamar Mandi (JKM): "))
    GRS = int(input("Kapasitas mobil dalam garasi (GRS): ")) + 1

    data_input = pd.DataFrame({
        'LT': [LT],
        'LB': [LB],
        'KT': [JKT],
        'KM': [JKM],
        'GRS': [GRS]
    })
    # data_input = min_max_input(data_input, data_mm)
    data_input = min_max_input(log(data_input), data_mm)
    # data_input = min_max_input(cb_root(data_input), data_mm)

    return data_input

def evaluate_model(model, X_test, y_test):
    start_time = time.time()
    y_pred = model.predict(X_test)
    end_time = time.time()
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    execution_time = end_time - start_time

    return mse, r2, execution_time

def normalize_user_input(user_input, data_mm):
    # Normalisasi input pengguna menggunakan data_mm yang sudah ada
    for fitur in user_input.columns:
        user_input[fitur] = (user_input[fitur] - data_mm[fitur][0]) / (data_mm[fitur][1] - data_mm[fitur][0])
    return user_input

## Dataset Harga Rumah


Link google sheets dataset
Klik File -> Download -> Comma Sepperated Values (.csv) -> save

https://docs.google.com/spreadsheets/d/1Damcm60bEj0Ar7RbRuPz9rl6yj-gNp9kTD_vM1RDR8A/edit#gid=1870171427

Untuk row Garasi 1 itu artinya ADA dan 0 itu TIDAK ADA

Untuk row Kota 1 itu artinya JAKSEL


In [ ]:
data_raw = pd.read_csv('https://drive.google.com/uc?id=1fpbEXvCH_SqdLGsmy5dXhZFskb89wVnt', delimiter=";", encoding="windows-1252")
data_raw = data_raw.drop(['NAMA RUMAH', 'NO'], axis=1)
data_raw['GRS'] = data_raw['GRS'] + 1

data_raw

,HARGA,LB,LT,KT,KM,GRS
0,3800000000,220,220,3,3,1
1,4600000000,180,137,4,3,3
2,3000000000,267,250,4,4,5
3,430000000,40,25,2,2,1
4,9000000000,400,355,6,5,4
...,...,...,...,...,...,...
1005,9000000000,450,550,10,10,4
1006,4000000000,160,140,4,3,3
1007,4000000000,139,230,4,4,2
1008,19000000000,360,606,7,4,1


#Normalisasi Data

In [ ]:
def min_max(df:pd.DataFrame):
    list_fitur = df.columns[:]
    data_mm = {fitur:v for (fitur, v) in zip(list_fitur, [[] for i in range(len(list_fitur))])}
    for fitur in list_fitur:
        max = df[fitur].max()
        min = df[fitur].min()
        data_mm[fitur] = [min, max]
        df[fitur] = (df[fitur]-min)/(max-min)
    return df, data_mm

def min_max_input(df_input, data_mm):
    list_fitur = df_input.columns[:]

    for fitur in list_fitur:
        df_input[fitur] = (df_input[fitur] - data_mm[fitur][0]) / (data_mm[fitur][1] - data_mm[fitur][0])
    return df_input

def log(df):
    list_fitur = df.columns[:]
    for fitur in list_fitur:
        df[fitur] = np.log(df[fitur])
    return df

def cb_root(df):
    list_fitur = df.columns[:]
    for fitur in list_fitur:
        df[fitur] = np.power(df[fitur], (1/3))
    return df

data, data_mm = min_max(log(data_raw.copy()))
data

,HARGA,LB,LT,KT,KM,GRS
0,0.434200,0.510779,0.540264,0.251930,0.477121,0.000000
1,0.472271,0.450654,0.422598,0.430677,0.477121,0.458157
2,0.387095,0.568792,0.572021,0.430677,0.602060,0.671188
3,0.000000,0.000000,0.000000,0.000000,0.301030,0.000000
4,0.606014,0.689903,0.659133,0.682606,0.698970,0.578130
...,...,...,...,...,...,...
1005,0.606014,0.725194,0.767894,1.000000,1.000000,0.578130
1006,0.444421,0.415363,0.427979,0.430677,0.477121,0.458157
1007,0.444421,0.373207,0.551307,0.430677,0.602060,0.289065
1008,0.754910,0.658335,0.791982,0.778385,0.602060,0.000000


#Training

In [ ]:
data = np.array(data)

# Pisahkan fitur dan target
X = data[:, 1:]  # Fitur
y = data[:, 0]   # Harga

# Bagi data menjadi data latih dan data uji (70% latih, 30% uji)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Inisialisasi model ELM
input_size = X_train.shape[1]
hidden_size = 20  # Jumlah neuron pada hidden layer, bisa disesuaikan
output_size = 1   # Satu output untuk prediksi harga

elm_model = ELM(input_size, hidden_size, output_size)

# Training model
elm_model.train(X_train, y_train)

#Testing

In [ ]:
# Evaluasi model
mse, r2, execution_time = evaluate_model(elm_model, X_test, y_test)

# Cetak hasil evaluasi
print("Mean Squared Error on Test Data:", mse)
print("R2 Score on Test Data:", r2)
print("Execution Time:", execution_time, "seconds")

Mean Squared Error on Test Data: 0.00433779736031069
R2 Score on Test Data: 0.8136615596860727
Execution Time: 0.0011572837829589844 seconds


#Prediksi Harga Rumah

In [ ]:
prediksi = elm_model.predict(X_test)
harga_min = min(data_mm['HARGA'])
harga_max = max(data_mm['HARGA'])
prediksi = prediksi * (harga_max - harga_min) + harga_min
sum(prediksi < 0)

# # Input data pengguna
user_input = get_user_input(data_mm)
# Prediksi harga rumah berdasarkan input pengguna
predicted_price = elm_model.predict(user_input)
predicted_price = predicted_price * (harga_max - harga_min) + harga_min
# predicted_price = np.power(predicted_price, 3)
predicted_price = np.exp(predicted_price)
formatted_price = format_rupiah(predicted_price[0])
print("Estimasi Harga Rumah:", formatted_price)

Masukkan fitur-fitur rumah sesuai kebutuhan anda
Luas Tanah (LT): 150
Luas Bangunan (LB): 120
Jumlah Kamar Tidur (JKT): 3
Jumlah Kamar Mandi (JKM): 2
Kapasitas mobil dalam garasi (GRS): 1
Estimasi Harga Rumah: Rp 2,598,324,380.78
